In [ ]:
%%javascript
require("notebook/js/notebook").Notebook.prototype.scroll_to_bottom = function () {}

In [ ]:
from notebook_prelude import *

## Datasets statistics (Text and Graph)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

preprocessor = transformers.SimplePreProcessingTransformer()
trans = transformers.NxGraphToTupleTransformer()

UNDIRECTED = True
LIMIT_DATASET = None
filtered_datasets = []
for dataset in dataset_helper.get_dataset_names_with_concept_map():
    if LIMIT_DATASET and dataset not in LIMIT_DATASET: continue
    concept_map = dataset_helper.get_all_cached_graph_datasets(dataset_name=dataset, graph_type=TYPE_CONCEPT_MAP)[0]
    cooccurence_graphs = [x for x in dataset_helper.get_all_cached_graph_datasets(dataset, graph_type=TYPE_COOCCURRENCE) if '_cooccurrence_1' in x and '_all_' in x]
    if not len(cooccurence_graphs):
        print('WARNING: No co-occurrence graphs found for: {}'.format(dataset))
        continue
    
    filtered_datasets.append((dataset, concept_map, cooccurence_graphs[0]))

data = collections.defaultdict(lambda: [])
for dataset, concept_map, cooccurence_graph in helper.log_progress(filtered_datasets):
    print('Dataset: {}'.format(dataset))

    X, Y = dataset_helper.get_dataset(dataset)
    # Add general dataset statistics
    data['dataset'].append(dataset)
    data['num_documents'].append(len(X))
    data['num_classes'].append(len(set(Y)))

    print('\t', 'text')
    X = preprocessor.fit_transform(X)

    # Add text statistics
    count_vec = CountVectorizer()
    doc_vecs = count_vec.fit_transform(X)
    
    all_words = set(count_vec.vocabulary_.keys())
    data['document_lengths'].append([len(x) for x in X])
    data['num_words'].append(doc_vecs.sum())
    data['num_unique_words'].append(len(all_words))
    data['median_doc_length'].append(np.median([len(x) for x in X]))
    data['median_words_per_doc'].append(np.median(np.squeeze(np.asarray(doc_vecs.sum(axis = 1)))))
    
    # Add graph statistics
    for graph_type, graph_cache_file in [(TYPE_CONCEPT_MAP, concept_map), (TYPE_COOCCURRENCE, cooccurence_graph)]:
        print('\t', graph_type)
        X_graph, _ = dataset_helper.get_dataset_cached(graph_cache_file)
        X_graph = graph_helper.get_graphs_only(X_graph)
        
        if UNDIRECTED:
            X_graph = [nx.Graph(x) for x in X_graph]
        
        all_labels = [x.nodes() for x in X_graph]
        all_labels_flat = list(chain.from_iterable(all_labels))
        num_labels = len(all_labels_flat)
        all_labels_unique = set(all_labels_flat)
        num_edges = sum([nx.number_of_edges(x) for x in X_graph])
        
        data['num_nodes_{}'.format(graph_type)].append(num_labels)
        data['num_unique_nodes_labels_{}'.format(graph_type)].append(len(all_labels_unique))
        data['num_edges_{}'.format(graph_type)].append(num_edges)

In [ ]:
with open('tmp/dataset_statistics.npy', 'wb') as f:
    pickle.dump(dict(**data), f)

## Graph statistics, eg. ratio_edges_to_nodes

In [ ]:
df = pd.DataFrame(data).set_index('dataset')
df['ratio_unique_words'] = df.num_unique_words / df.num_nodes_cooccurrence

for graph_type in [TYPE_CONCEPT_MAP, TYPE_COOCCURRENCE]:
    df['ratio_edges_to_nodes_{}'.format(graph_type)] = df['num_edges_{}'.format(graph_type)] / df['num_nodes_{}'.format(graph_type)]
    df['ratio_nodes_to_words_{}'.format(graph_type)] = df['num_nodes_{}'.format(graph_type)] / df.num_words

    
df['ratio_unique_words'] = df.num_unique_words / df.num_nodes_cooccurrence
df['ratio_ratio_ratio_edges_to_nodes_cmap_to_coo'] = df['ratio_edges_to_nodes_concept_map'] / df['ratio_edges_to_nodes_cooccurrence']
for col in ['median_doc_length', 'median_words_per_doc']:
    df[col] = df[col].astype(int)

df = df[sorted(df.columns)]

### Nodes per graph type

In [ ]:
for t in GRAPH_TYPES:
    df['nodes_per_graph_{}'.format(t)] = df['num_nodes_{}'.format(t)] / df.num_documents

### Create latex tables for text and graphs

In [ ]:
df['num_words_k'] = (df.num_words / 1000).map(lambda x: '{:,}k'.format(int(x)).replace(',', '.')) 
df['num_documents_k'] = (df.num_documents).map(lambda x: '{:,}'.format(int(x)).replace(',', '.')) 
df['nodes_per_graph_concept_map_str'] = (df.num_words).map(lambda x: '{:.0f}'.format(x)) 
df['nodes_per_graph_cooccurrence_str'] = (df.num_words).map(lambda x: '{:.0f}'.format(x)) 

In [ ]:
RENAME_COLS = {
    'dataset': {
        'num_classes': '# classes', 
        'num_documents_k': '# docs', 
        'num_words_k': '# words',
        'median_words_per_doc': 'median #words/doc',
        'ratio_unique_words': '#uniq. words/#words',
    },
    'graphs': {
        'ratio_nodes_to_words_concept_map': '#nodes/#words cmap',
        'ratio_nodes_to_words_cooccurrence': '#nodes/#words coo',
        'ratio_edges_to_nodes_concept_map': '#edges/#nodes cmap', 
        'ratio_edges_to_nodes_cooccurrence': '#edges/#nodes coo', 
        'nodes_per_graph_concept_map': '#nodes/graph cmap',
        'nodes_per_graph_cooccurrence': '#nodes/graph coo',
    }
}

df.loc['mean'] = df.mean()

no_float_cols = ['nodes_per_graph_concept_map']

for key, rename_cols in RENAME_COLS.items():
    df_ = df
    if key == 'dataset':
        df_ = df_[df_.index !='mean']
    df_stats = df_[list(rename_cols.keys())].rename(columns = rename_cols)
    print(df_stats.to_latex(float_format = '%.2f').replace('.0 &', ' &'))
    display(df_stats)

In [ ]:
with open('tmp/dataset_statistics.npy', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
with open('tmp/dataset_statistics.npy', 'rb') as f:
    df = pickle.load(f)

## Co-Occurrence graph statistics

### All-words vs. only-nouns compression

In [ ]:
WINDOW_SIZE = 1
datasets_coo = collections.defaultdict(lambda: {})

cooccurrence_graph_files = [x for x in dataset_helper.get_all_cached_graph_datasets() if 'cooccurrence' in x]

cmap_datasets = dataset_helper.get_dataset_names_with_concept_map()
for graph_cf in cooccurrence_graph_files:
    dataset = filename_utils.get_dataset_from_filename(graph_cf)
    if dataset not in cmap_datasets: continue
    window_size = filename_utils.get_cooccurrence_window_size_from_filename(graph_cf)
    words = filename_utils.get_cooccurrence_words_from_filename(graph_cf)
    if window_size != WINDOW_SIZE: continue
    if dataset in datasets_coo and words in datasets_coo[dataset]: continue
    print(graph_cf)
    X, Y = dataset_helper.get_dataset_cached(graph_cf)
    all_labels = graph_helper.get_all_node_labels(X)
    datasets_coo[dataset][words] = all_labels

In [ ]:
data_ = collections.defaultdict(lambda: [])
for dataset, words in datasets_coo.items():
    for word, labels in words.items():
        data_['dataset'].append(dataset)
        data_['word'].append(word.replace('-', '_').replace('all', 'all_words'))
        data_['label_count'].append(len(labels))
    
df = pd.DataFrame(data_)
df = df.pivot(index = 'dataset', columns = 'word', values = 'label_count')
df['word_ratio'] = df.only_nouns / df.all_words
df

## Concept map

### Recurring concepts

In [ ]:
all_labels = collections.defaultdict(lambda: {})
dataset = 'ling-spam'
dataset = None
for gcf in dataset_helper.get_all_cached_graph_datasets(graph_type=TYPE_CONCEPT_MAP):
    dataset = filename_utils.get_dataset_from_filename(gcf)
    graph_type = graph_helper.get_graph_type_from_filename(gcf)

    if graph_type in all_labels[dataset]: continue
    
    print('{:30} {}'.format(dataset, graph_type))
    X, Y = dataset_helper.get_dataset_cached(gcf)
    X = graph_helper.convert_graphs_to_adjs_tuples(X, copy = True)
    all_labels[dataset][graph_type] = [labels for _, labels in X]

In [ ]:
data_ = collections.defaultdict(lambda: [])
for dataset, graph_types in all_labels.items():
    for graph_type, labels in graph_types.items():
        labels_flat = list(chain.from_iterable(labels))
        c = collections.Counter(labels_flat)
        df_cmap_occurrences = pd.DataFrame(list(c.items()), columns = ['label', 'occurrences'])
        data_['dataset'] += [dataset] * len(c.keys())
        data_['label'] += c.keys()
        data_['occurrences'] += c.values()
        data_['num_docs'] += [len(labels)] * len(c.keys())
        data_['type'] += [graph_type] * len(c.keys())
df_occs = pd.DataFrame(data_)

In [ ]:
df_occ_only_once = (
    (
        df_occs[df_occs.occurrences == 1].groupby(['dataset', 'type']).occurrences.value_counts()
        / 
        df_occs.groupby('dataset').occurrences.sum()
    ).groupby(['dataset', 'type']).sum()
).unstack()

df_occ_only_once = df_occ_only_once[pd.isna(df_occ_only_once[TYPE_CONCEPT_MAP]) == False]
print(df_occ_only_once.to_latex())
df_occ_only_once

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 9999999999]


data = collections.defaultdict(list)
for (dataset, graph_type), df_ in df_occs.groupby(['dataset', 'type']):
    print(dataset, graph_type)
    bins_ = pd.cut(df_.occurrences, bins=bins)
    val_counts = bins_.value_counts()
    sum_ = val_counts.sum()
    for bin_, val in val_counts.iteritems():
        data['ratio'].append(val / sum_ * 100)
        data['dataset'].append(dataset)
        data['type'].append(graph_type)
        data['bin'].append(bin_)

df____ = pd.DataFrame(data)
#
#.plot(kind='barh', stacked=True)

In [ ]:
df__ = df____.pivot(index='dataset', columns='bin', values='ratio').sort_index(ascending=False)

max_column = np.sort(df__.columns.map(lambda x: x.right))

def get_columns_name(x):
    val = x.right
    is_too_big = val > 999
    eq_sign = '\geq' if is_too_big else '='
    val = max_column[-2] + 1 if is_too_big else val
    return '$|n_v|{}{}$'.format(eq_sign, val)
    
df__.rename(columns=get_columns_name, inplace=True)
df__.columns.name = None
fig, ax = plt.subplots(figsize=(8, 2.3))
df__.plot(kind='barh', stacked=True, ax = ax)
ax.set_xlim((0, 118))
ax.grid(False)
ax.set_xlabel('%')
fig.tight_layout()
save_fig(fig, 'percentage_distribution_concept_occurrences')

In [ ]:
fig, ax = plt.subplots()
last = 200#df_occs.occurrences.quantile(q=0.98)
df_occs[df_occs.occurrences < last].groupby('dataset').occurrences.hist(bins = 120, ax = ax, alpha = 0.2)
ax.set_yscale('log')
ax.legend()